<a href="https://colab.research.google.com/github/simulate111/Introduction-to-Human-Language-Technology/blob/main/intro_2023_exercise_10_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating word embeddings

This notebook demonstrates how to create new `word2vec` word embeddings using `gensim`.

---

## Setup

We'll use [`gensim`](https://pypi.org/project/gensim/) to induce the word embeddings and (following the [sentence splitting and tokenization notebook](https://github.com/TurkuNLP/intro-to-nlp/blob/master/sentence_splitting_and_tokenization.ipynb)) the [`sentence-splitter`](https://pypi.org/project/sentence-splitter/) package to split sentences.

In [1]:
!pip install --quiet gensim sentence-splitter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 607.2 kB/s eta 0:00:00


In [2]:
!wget -nc http://dl.turkunlp.org/TKO_7095_2023/fiwiki-20221120-sample.txt

--2024-04-17 23:21:16--  http://dl.turkunlp.org/TKO_7095_2023/fiwiki-20221120-sample.txt
Resolving dl.turkunlp.org (dl.turkunlp.org)... 195.148.30.23
Connecting to dl.turkunlp.org (dl.turkunlp.org)|195.148.30.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 500364237 (477M) [text/plain]
Saving to: ‘fiwiki-20221120-sample.txt’

fiwiki-20221120-sam 100%[===================>] 477.18M  14.1MB/s    in 30s     

2024-04-17 23:21:47 (15.7 MB/s) - ‘fiwiki-20221120-sample.txt’ saved [500364237/500364237]



In [3]:
import regex

from sentence_splitter import SentenceSplitter

## Download and read data

We'll use the English data here, but you can switch out to the Finnish by commenting out the lines loading the Finnish and uncommenting the lines loading the English data.

In [4]:
# !wget -nc http://dl.turkunlp.org/TKO_7095_2023/fiwiki-20221120-sample.txt
!wget -nc http://dl.turkunlp.org/TKO_7095_2023/enwiki-20220301-sample.txt

--2024-04-17 23:21:47--  http://dl.turkunlp.org/TKO_7095_2023/enwiki-20220301-sample.txt
Resolving dl.turkunlp.org (dl.turkunlp.org)... 195.148.30.23
Connecting to dl.turkunlp.org (dl.turkunlp.org)|195.148.30.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 527260381 (503M) [text/plain]
Saving to: ‘enwiki-20220301-sample.txt’

enwiki-20220301-sam 100%[===================>] 502.83M  16.1MB/s    in 32s     

2024-04-17 23:22:20 (15.6 MB/s) - ‘enwiki-20220301-sample.txt’ saved [527260381/527260381]



In [5]:
#paragraphs = open('fiwiki-20221120-sample.txt').readlines()
paragraphs = open('enwiki-20220301-sample.txt').readlines()

Check what we have

In [6]:
print('Total paragraphs:', len(paragraphs))
print('Total characters:', sum(len(p) for p in paragraphs))

Total paragraphs: 1000000
Total characters: 526082203


So, that's a million paragraphs totaling over 500 million characters. This is a reasonably large corpus, though still notably smaller than corpora on which word2vec models are generally trained.

In [7]:
for i, p in enumerate(paragraphs[:10]):
    print(f'paragraph {i}: {p}', end='')

paragraph 0: Incumbent CM Mayawati began her campaign on 27 January at a rally in Bijnor. On 15 January, she released the BSP's list of candidates for all the 403 constituencies. The list included 88 candidates belonging to SCs, 113 from OBCs, 85 religious minorities and 117 upper castes, out of which 74 are Brahmins.
paragraph 1: As a member of the OSS Research and Analysis Division, Wheeler had government security clearance to received secret and confidential "ditto" copies of monthly and semi-monthly reports of political developments throughout the world. Wheeler is alleged to have passed these reports as well as handwritten and typewritten material of cable reports from the State Department and the OSS to Soviet intelligence.  Wheeler is alleged to have provided information on the organization and policies of British intelligence services and furnished memoranda prepared by the Foreign Nationalities Branch of OSS on material relating to the particular racial groups and activities w

## Split sentences and tokenize

Following the [sentence splitting and tokenization notebook](https://github.com/TurkuNLP/intro-to-nlp/blob/master/sentence_splitting_and_tokenization.ipynb)).

(The line `%%time` is Jupyter ["cell magic"](https://ipython.readthedocs.io/en/stable/interactive/magics.html) to print execution time.)

In [8]:
%%time

splitter = SentenceSplitter(language='fi')

sentences = [s for p in paragraphs for s in splitter.split(p)]

CPU times: user 19min 4s, sys: 4.62 s, total: 19min 8s
Wall time: 19min 48s


In [9]:
%%time

TOKENIZE_RE = regex.compile(r'([[:alnum:]]+|\S)')

tokenized = [TOKENIZE_RE.findall(s) for s in sentences]

CPU times: user 1min 38s, sys: 8.93 s, total: 1min 47s
Wall time: 1min 49s


So, with this approach, total time for sentence-splitting was ~15 min, and tokenization was just over one minute for the million paragraphs.

See a few examples

In [10]:
for i, s in enumerate(tokenized[:10]):
    print(f'Sentence {i}:', s)

Sentence 0: ['Incumbent', 'CM', 'Mayawati', 'began', 'her', 'campaign', 'on', '27', 'January', 'at', 'a', 'rally', 'in', 'Bijnor', '.']
Sentence 1: ['On', '15', 'January', ',', 'she', 'released', 'the', 'BSP', "'", 's', 'list', 'of', 'candidates', 'for', 'all', 'the', '403', 'constituencies', '.']
Sentence 2: ['The', 'list', 'included', '88', 'candidates', 'belonging', 'to', 'SCs', ',', '113', 'from', 'OBCs', ',', '85', 'religious', 'minorities', 'and', '117', 'upper', 'castes', ',', 'out', 'of', 'which', '74', 'are', 'Brahmins', '.']
Sentence 3: ['As', 'a', 'member', 'of', 'the', 'OSS', 'Research', 'and', 'Analysis', 'Division', ',', 'Wheeler', 'had', 'government', 'security', 'clearance', 'to', 'received', 'secret', 'and', 'confidential', '"', 'ditto', '"', 'copies', 'of', 'monthly', 'and', 'semi', '-', 'monthly', 'reports', 'of', 'political', 'developments', 'throughout', 'the', 'world', '.']
Sentence 4: ['Wheeler', 'is', 'alleged', 'to', 'have', 'passed', 'these', 'reports', 'as', 

## Alternative using UDpipe

You can alternatively perform sentence splitting and tokenization using [UDPipe](https://lindat.mff.cuni.cz/services/udpipe/) as follows, but note that this is considerably slower.

In [11]:
!pip3 install --quiet ufal.udpipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 936.8/936.8 kB 4.8 MB/s eta 0:00:00


In [12]:
!wget -nc https://github.com/TurkuNLP/intro-to-nlp/raw/master/Data/fi.segmenter.udpipe
!wget -nc https://github.com/TurkuNLP/intro-to-nlp/raw/master/Data/en.segmenter.udpipe

--2024-04-17 23:44:12--  https://github.com/TurkuNLP/intro-to-nlp/raw/master/Data/fi.segmenter.udpipe
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/TurkuNLP/intro-to-nlp/master/Data/fi.segmenter.udpipe [following]
--2024-04-17 23:44:12--  https://raw.githubusercontent.com/TurkuNLP/intro-to-nlp/master/Data/fi.segmenter.udpipe
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22607316 (22M) [application/octet-stream]
Saving to: ‘fi.segmenter.udpipe’

fi.segmenter.udpipe 100%[===================>]  21.56M  --.-KB/s    in 0.1s    

2024-04-17 23:44:13 (149 MB/s) - ‘fi.segmenter.udpipe’ saved [226073

In [13]:
import ufal.udpipe as udpipe

#model = udpipe.Model.load('fi.segmenter.udpipe')
model = udpipe.Model.load('en.segmenter.udpipe')

pipeline = udpipe.Pipeline(model, 'tokenize', 'none', 'none', 'horizontal')

(We're only processing the first 10,000 paragraphs here as this is quite slow.)

In [14]:
%%time

udpipe_segmented = [pipeline.process(p) for p in paragraphs[:10000]]

CPU times: user 3min 49s, sys: 677 ms, total: 3min 50s
Wall time: 4min 6s


For UDPipe, the sentence splitting and tokenization took over three minutes for just 10,000 paragraphs.

In [15]:
udpipe_sentences = [s for t in udpipe_segmented for s in t.split('\n')]
udpipe_tokenized = [s.split() for s in udpipe_sentences]

In [16]:
for i, s in enumerate(udpipe_tokenized[:10]):
    print(f'UDpipe sentence {i}:', s)

UDpipe sentence 0: ['Incumbent', 'CM', 'Mayawati', 'began', 'her', 'campaign', 'on', '27', 'January', 'at', 'a', 'rally', 'in', 'Bijnor', '.']
UDpipe sentence 1: ['On', '15', 'January', ',', 'she', 'released', 'the', 'BSP', "'s", 'list', 'of', 'candidates', 'for', 'all', 'the', '403', 'constituencies', '.']
UDpipe sentence 2: ['The', 'list', 'included', '88', 'candidates', 'belonging', 'to', 'SCs', ',', '113', 'from', 'OBCs', ',', '85', 'religious', 'minorities', 'and', '117', 'upper', 'castes', ',', 'out', 'of', 'which', '74', 'are', 'Brahmins', '.']
UDpipe sentence 3: []
UDpipe sentence 4: ['As', 'a', 'member', 'of', 'the', 'OSS', 'Research', 'and', 'Analysis', 'Division', ',', 'Wheeler', 'had', 'government', 'security', 'clearance', 'to', 'received', 'secret', 'and', 'confidential', '"', 'ditto', '"', 'copies', 'of', 'monthly', 'and', 'semi-monthly', 'reports', 'of', 'political', 'developments', 'throughout', 'the', 'world', '.']
UDpipe sentence 5: ['Wheeler', 'is', 'alleged', 'to',

## Create word2vec model

We'll create a word2vec model using `gensim` mostly with default parameters. For details on the parameters of this class, see <https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec>

In [17]:
%%time

from gensim.models import Word2Vec

model = Word2Vec(
    sentences=tokenized,
    vector_size=100,
)

CPU times: user 26min 59s, sys: 7.63 s, total: 27min 7s
Wall time: 20min 59s


Test with Finnish words (if you've trained a Finnish model)

In [18]:
# for word in ('hyvä', 'huono', 'koira', 'kissa', 'kuningas', 'kuningatar'):
#     print(f'{word}:\t', [w for w, s in model.wv.most_similar(word)])

Test with English words (if you've trained an English model)

In [19]:
for word in ('good', 'bad', 'dog', 'cat', 'king', 'queen'):
    print(f'{word}:\t', [w for w, s in model.wv.most_similar(word)])

good:	 ['bad', 'decent', 'nice', 'perfect', 'great', 'poor', 'wonderful', 'little', 'genuine', 'quiet']
bad:	 ['good', 'terrible', 'horrible', 'tough', 'miserable', 'careless', 'sad', 'crazy', 'nice', 'poor']
dog:	 ['cat', 'rabbit', 'puppy', 'donkey', 'pet', 'fox', 'goat', 'monkey', 'cow', 'dogs']
cat:	 ['dog', 'rabbit', 'monkey', 'snake', 'spider', 'creature', 'wolf', 'beast', 'goat', 'doll']
king:	 ['prince', 'emperor', 'sultan', 'monarch', 'ruler', 'duke', 'queen', 'Emperor', 'kings', 'throne']
queen:	 ['princess', 'prince', 'king', 'consort', 'bride', 'empress', 'monarch', 'dowager', 'countess', 'bridegroom']


Not too bad a result for a half-hour run! Some observations:

* Perhaps surprisingly, antonyms (words with opposite meaning) show up in the lists of most similar words. This is a known issue with word embeddings (see e.g. [Scheible _et al._ 2013](https://aclanthology.org/I13-1056.pdf)) and makes sense if you think about it: for example, _bad_ can appear in many of the same contexts as _good_.
* These word embeddings group animal words with both _dog_ and _cat_, but don't show the level of granularity we got with the embeddings trained on more data, where different types of dogs and cats were more similar to the words _dog_ and _cat_ (respectively) than other animals.
* We see a similar phenomenon with _king_ and _queen_ as with animals, with many words related to royalty and ruling classes showing up in both, with more limited distinction by e.g. gender than found in embeddings trained on more data.